<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #4: Add new clips to a Zooniverse workflow</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Nov 8th, 2021</h5>

# Set up and requirements

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import utils.tutorials_utils as t_utils
import utils.t4_utils as t4

print("Packages loaded successfully")

### Choose your project

In [ ]:
project = t_utils.choose_project()

### Initiate SQL database and populate sites, movies and species

In [ ]:
# Initiate the SQL database 
%run -i "../db_starter/starter.py" --project_name $project.value

# Retrieve info about zooniverse clips

In [ ]:
zoo_clips_info_dict, zoo_project = t4.retrieve_zoo_clips_info(project_name = project.value, 
                                                              zoo_info = ["subjects"])

# Retrieve info about movies hosted in the server

In [ ]:
# Choose the server where the movies are hosted
server_i = t_utils.choose_server()

In [ ]:
# Store info about the movies available in the server
available_movies_df, server_i_dict = t4.retrieve_movie_info_from_server(server_i = server_i.value)

### Select the movie you want to upload to Zooniverse

In [ ]:
movie_i = t4.movie_to_upload(available_movies_df = available_movies_df)

### Check if movie is already in Zooniverse

Remember to query the newest zooniverse data to get the most up to date list of clips uploaded

In [ ]:
# Check movie hasn't been uploaded to Zooniverse
t4.check_movie_uploaded(project_name = project.value, 
                        movie_i = movie_i.value)

### Specify the number of clips and clip length

In [ ]:
clip_selection = t4.select_clip_n_len(movie_i = movie_i.value)

### Review the clips that will be created

In [ ]:
t4.review_clip_selection(clip_selection = clip_selection, 
                         movie_i = movie_i.value)

### Create the clips

In [ ]:
clips_to_upload_df = t4.create_clips(available_movies_df = available_movies_df, 
                                     movie_i = movie_i.value, 
                                     clip_selection = clip_selection, 
                                     server = server_i.value, 
                                     project_name = project.value)

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

### Review clips

In [ ]:
t4.check_clip_size(clip_paths = clips_to_upload_df["clip_path"].unique())

### Video modifications

In [ ]:
# Work in progress
clip_modification = t4.select_modification()

In [ ]:
# Write the clip modification details. See ffmpeg documentation for instructions (http://ffmpeg.org/ffmpeg.html)
modification_details = {
  "-crf": "30",
  "-c:v": "libx264",
  "-vf": "curves=red=0/0 0.396/0.67 1/1:green=0/0 0.525/0.451 1/1:blue=0/0 0.459/0.517 1/1,scale=1280:-1",#borrowed from https://www.element84.com/blog/color-correction-in-space-and-at-sea                         
  "-pix_fmt": "yuv420p",
  "-preset": "veryfast"
}

In [ ]:
# Modify the clips
clips_ready_df = t4.modify_clips(clips_to_upload_df = clips_to_upload_df[175:179].reset_index(drop=True), 
                                 clip_modification = clip_modification.value, 
                                 modification_details = modification_details)

In [ ]:
# Review the size of the modified clips
t4.check_clip_size(clip_paths = clips_ready_df["modif_clip_path"].unique())

In [ ]:
# Compare the original and modified clips
t4.compare_clips(df = clips_ready_df)

### Set Zooniverse metadata

In [ ]:
upload_to_zoo, sitename, created_on = t4.set_zoo_metadata(df = clips_ready_df,
                                                          project_name = project.value)
upload_to_zoo = upload_to_zoo.fillna("unknown")

### Upload clips to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
t4.upload_clips_to_zooniverse(upload_to_zoo = upload_to_zoo, 
                              sitename = sitename,
                              created_on = created_on,
                              project = project)

In [ ]:
#END